In [ ]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from autogluon.tabular import TabularPredictor, TabularDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
df = pd.read_csv("Data Sheet 4.csv")
train_set = pd.read_csv("Data Sheet 2.csv")
test_set = pd.read_csv("Data Sheet 3.csv")

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
print(train_set.shape)
print(test_set.shape)

In [ ]:
train_set = train_set.drop(columns = ['Unnamed: 0'])
test_set = test_set.drop(columns = ['Unnamed: 0'])

In [ ]:
train_set.head()

#### Polynomialize Features

In [ ]:
exclude_columns = ['year', 'country', 'gini']

In [ ]:
train_set_poly = train_set.drop(columns=exclude_columns)
test_set_poly = test_set.drop(columns=exclude_columns)
train_set_untouched = train_set[exclude_columns]
test_set_untouched = test_set[exclude_columns] 

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
train_set_poly_transformed = poly.fit_transform(train_set_poly)
test_set_poly_transformed = poly.fit_transform(test_set_poly)

In [ ]:
train_set_poly_transformed = pd.DataFrame(train_set_poly_transformed, columns=poly.get_feature_names_out(train_set_poly.columns))
test_set_poly_transformed = pd.DataFrame(test_set_poly_transformed, columns=poly.get_feature_names_out(test_set_poly.columns))

In [ ]:
train_set_final = pd.concat([train_set_poly_transformed, train_set_untouched.reset_index(drop=True)], axis=1)
test_set_final = pd.concat([test_set_poly_transformed, test_set_untouched.reset_index(drop=True)], axis=1)

#### AutoGluon

In [ ]:
label = 'gini'
train_set[label].describe()

In [ ]:
predictor = TabularPredictor(label=label, problem_type='regression').fit(train_set_final)

In [ ]:
y_pred = predictor.predict(test_set_final.drop(columns=[label]))
y_pred.head()

In [ ]:
predictor.evaluate(test_set_final, silent=True)

In [ ]:
predictor.leaderboard(test_set_final)